In [1]:
import pandas as pd
import numpy as np
import sys
from math import sqrt
sys.path.append('../..')
from modules import utils
import gpflow
from gpflow import set_trainable
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import warnings
import tensorflow as tf
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
%matplotlib inline

2023-08-15 02:31:06.845788: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-15 02:31:06.909879: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-15 02:31:06.911907: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-08-15 02:31:06.911916: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudar

In [2]:
SEED = 42
# random.seed(SEED)
np.random.seed(SEED)
# os.environ['PYTHONHASHSEED']=str(SEED)
tf.random.set_seed(SEED)
# gpflow.config.set_default_seed(SEED)

In [3]:
kampala_devices = pd.read_csv('../data/kampala_devices.csv', usecols=['lat', 'long', 'id'])
kampala_devices.head()

,id,lat,long
0,930434,0.360209,32.610756
1,718028,0.307500,32.620600
2,912224,0.346460,32.703280
3,930426,0.365500,32.646800
4,930427,0.268900,32.588000


In [4]:
len(kampala_devices)

37

In [5]:
kampala_df = pd.read_csv('../data/kampala_data_with_met.csv', parse_dates=['timestamp'])
kampala_df.head()

,timestamp,pm2_5_calibrated_value,pm2_5_raw_value,pm10_raw_value,pm10_calibrated_value,site_id,device_number,site_name,latitude,longitude,city,temperature,humidity,wind_speed,precipitation,wind_direction,wind_gusts
0,2021-09-01 00:00:00+00:00,33.22,55.88,75.45,NaN,60d2affe7e9018a1a8d38c10,0,"Luzira, Nakawa",0.299092,32.647930,Kampala,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-09-01 00:00:00+00:00,33.35,72.55,87.86,NaN,60d2b0317e9018a1a8d38c11,0,Rubaga,0.343729,32.551314,Kampala,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-09-01 00:00:00+00:00,47.90,73.22,83.57,NaN,60d2b0557e9018a1a8d38c12,0,Rubaga,0.329428,32.551303,Kampala,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-09-01 00:00:00+00:00,41.52,62.39,75.87,NaN,60d2b0807e9018a1a8d38c13,0,"Rubaga Div, Rubaga",0.305636,32.563312,Kampala,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-09-01 00:00:00+00:00,31.78,49.57,63.87,NaN,60d2b0a57e9018a1a8d38c14,0,KampalaCentral,0.319970,32.595548,Kampala,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
old_latitudes = kampala_df['latitude'].unique()
old_longitudes = kampala_df['longitude'].unique()
old_device_ids = kampala_df['device_number'].unique()
len(old_latitudes), len(old_longitudes), len(old_device_ids)

(73, 73, 57)

In [8]:
final_df = pd.DataFrame()
cols = ['timestamp', 'latitude', 'longitude', 'temperature', 'humidity', 'pm2_5_calibrated_value', 'device_number']
for i, device_id in kampala_devices.id.iteritems():
    device_df = utils.get_device_data(kampala_df, device_id, cols)
    processed_df = utils.preprocessing(device_df)
    final_df = pd.concat([final_df, processed_df])
final_df.reset_index(drop=True, inplace=True)
final_df.head()

,time,latitude,longitude,temperature,humidity,pm2_5,device_number
0,453031.0,0.360209,32.610756,24.233333,75.741667,10.5477,930434
1,453032.0,0.360209,32.610756,25.650000,68.066667,16.4250,930434
2,453033.0,0.360209,32.610756,26.150000,64.950000,17.7239,930434
3,453034.0,0.360209,32.610756,26.741667,63.066666,16.1533,930434
4,453035.0,0.360209,32.610756,27.400000,62.199999,18.0123,930434


In [9]:
latitudes = final_df['latitude'].unique()
longitudes = final_df['longitude'].unique()
device_ids = final_df['device_number'].unique()
len(latitudes), len(longitudes), len(device_ids)

(35, 35, 34)

In [10]:
final_df = final_df.drop(['device_number'], axis=1)
final_df.head()

,time,latitude,longitude,temperature,humidity,pm2_5
0,453031.0,0.360209,32.610756,24.233333,75.741667,10.5477
1,453032.0,0.360209,32.610756,25.650000,68.066667,16.4250
2,453033.0,0.360209,32.610756,26.150000,64.950000,17.7239
3,453034.0,0.360209,32.610756,26.741667,63.066666,16.1533
4,453035.0,0.360209,32.610756,27.400000,62.199999,18.0123


#### delete from here

#### end here

In [11]:
def cross_validation(final_df, idx, kernel_variance, lengthscales, likelihood_variance, trainable_kernel, 
                     trainable_variance, trainable_lengthscales):
    device_indices = final_df[final_df.latitude==latitudes[idx]].index
    device_df = kampala_df[kampala_df.device_number == device_ids[idx]]
#     assert(len(device_indices) == len(device_df)-device_df.pm2_5_calibrated_value.isna().sum())
    
    test_df = final_df.loc[device_indices]
#     assert(len(test_df.longitude.unique()) == 1)
    
    train_df = pd.concat([final_df, test_df]).drop_duplicates(keep=False)
#     assert(len(train_df.longitude.unique()) == len(longitudes)-1)
#     assert len(final_df) == len(test_df) + len(train_df)
    
    X_train = train_df.iloc[:, 0:-1]
    y_train = train_df.iloc[:, -1]
    X_train, y_train = np.array(X_train), np.array(y_train).reshape(-1, 1)
    if X_train.shape[0] > 39999:
        X_train = X_train[::6, :]
        y_train = y_train[::6, :]
#     print('printing x_train')
#     print(f'X_train shape:{X_train.shape}')
    
    X_test = test_df.iloc[:, 0:-1]
    y_test = test_df.iloc[:, -1]
    X_test, y_test = np.array(X_test), np.array(y_test).reshape(-1, 1)
    #to delete
    #X_train, y_train, X_test, y_test = X_train[:100, :], y_train[:100, :], X_test[:100, :], y_test[:100, :]
    
    if lengthscales == 'train_shape':
        lengthscales = np.ones(X_train.shape[1])
    
    if (lengthscales is None) & (kernel_variance is None):
        k = gpflow.kernels.RBF() + gpflow.kernels.Bias()
    elif lengthscales is None:
        k = gpflow.kernels.RBF(variance=kernel_variance) + gpflow.kernels.Bias()
    elif kernel_variance is None:
        k = gpflow.kernels.RBF(lengthscales=lengthscales) + gpflow.kernels.Bias()
    else:
        k = gpflow.kernels.RBF(lengthscales=lengthscales, variance=kernel_variance) + gpflow.kernels.Bias()
#     print('Training model .....................')    
    m = gpflow.models.GPR(data=(X_train, y_train), kernel=k, mean_function=None)
    if likelihood_variance is None:
        pass
    else:
        m.likelihood.variance.assign(likelihood_variance)
    set_trainable(m.kernel.kernels[0].variance, trainable_kernel)
    set_trainable(m.likelihood.variance, trainable_variance)
    set_trainable(m.kernel.kernels[0].lengthscales, trainable_lengthscales)
    
    #optimization
#     print('Optimizing model ...........................')
    opt = gpflow.optimizers.Scipy()
    def objective_closure():
        return - m.log_marginal_likelihood()
    
    opt_logs = opt.minimize(objective_closure,
                            m.trainable_variables,
                            options=dict(maxiter=100))

    #prediction
    mean, var = m.predict_f(X_test)
    
    rmse = sqrt(mean_squared_error(y_test, mean.numpy()))
    mape = mean_absolute_percentage_error(y_test, mean.numpy())
    return rmse, mape
    
#     return mean.numpy(), var.numpy(), Xtest, Ytest, round(rmse, 2)

#### The real work

In [12]:
# lengthscales = [[0.08, 0.08, 1], None, 'train_shape']
lengthscale = [1, 0.08, 0.08, 3, 5]
# lengthscale = None
# likelihood_variances = [400, 625, None]
likelihood_variance = 400
# kernel_variances = [625, 400, None]
kernel_variance  = 625
# trainable_kernels = [True, False]
trainable_kernel = True
# trainable_variances = [True, False]
trainable_variance = True
# trainable_lengthscales = [False, True]
trainable_lengthscale = False #only changed this

In [13]:
# count = 0
results_df = pd.DataFrame()
# for lengthscale in lengthscales:
#     for likelihood_variance in likelihood_variances:
#         for kernel_variance in kernel_variances:
#             for trainable_kernel in trainable_kernels:
#                 for trainable_variance in trainable_variances:
#                     for trainable_lengthscale in trainable_lengthscales:
#                         count+=1
rmse_list = []
mape_list = []
#                         print(f'EXPERIMENT {count}')
for i in range(len(latitudes)):
    try:
        rmse, mape = cross_validation(final_df, i, kernel_variance, lengthscale, likelihood_variance, 
                                trainable_kernel, trainable_variance, trainable_lengthscale)
        rmse_list.append(rmse)
        mape_list.append(mape)
        print(f'{device_ids[i]} successful')
    except Exception as e:
        print(e)
        print(f'{device_ids[i]} failed')
        break

mean_rmse = np.mean(rmse_list)
mean_mape = np.mean(mape_list)
results_dict= {'lengthscale':lengthscale, 'likelihood_variance':likelihood_variance, 
               'kernel_variance':kernel_variance, 'trainable_kernel':trainable_kernel, 
               'trainable_variance':trainable_variance, 'trainable_lengthscale':trainable_lengthscale, 
               'avg_rmse':mean_rmse, 'rmse_list':rmse_list, 'avg_mape':mean_mape, 'mape_list':mape_list}
print(results_dict)
results_df = results_df.append(results_dict, ignore_index=True)
results_df.to_csv(f'../results/basic_results_lengthscales_with_met.csv', index=False)               

2023-08-15 02:31:54.178767: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-08-15 02:31:54.178797: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: PL1207-PRO.paris.inria.fr
2023-08-15 02:31:54.178800: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: PL1207-PRO.paris.inria.fr
2023-08-15 02:31:54.178909: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 525.125.6
2023-08-15 02:31:54.178930: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: NOT_FOUND: could not find kernel module information in driver version file contents: "NVRM version: NVIDIA UNIX Open Kernel Module for x86_64  525.125.06  Release Build  (dvs-builder@U16-A23-14-1)  Tue May 30 05:12:48 UTC 2023
GCC version:  gcc ve

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


2023-08-15 02:37:02.558267: W tensorflow/core/kernels/linalg/cholesky_op.cc:56] Cholesky decomposition was not successful. Eigen::LLT failed with error code 1. Filling lower-triangular output with NaNs.


930434 successful


2023-08-15 02:48:09.982465: W tensorflow/core/kernels/linalg/cholesky_op.cc:56] Cholesky decomposition was not successful. Eigen::LLT failed with error code 1. Filling lower-triangular output with NaNs.


...

2023-08-15 08:23:59.610489: W tensorflow/core/kernels/linalg/cholesky_op.cc:56] Cholesky decomposition was not successful. Eigen::LLT failed with error code 1. Filling lower-triangular output with NaNs.


912219 successful


2023-08-15 08:36:00.790927: W tensorflow/core/kernels/linalg/cholesky_op.cc:56] Cholesky decomposition was not successful. Eigen::LLT failed with error code 1. Filling lower-triangular output with NaNs.


912221 successful
912225 successful
930429 successful
index 34 is out of bounds for axis 0 with size 34


IndexError: index 34 is out of bounds for axis 0 with size 34

In [14]:
rmse_list

[17.1614825151291,
 18.202070759489523,
 15.891526922176903,
 16.842808105242455,
 26.748096711824918,
 29.18152624380592,
 20.75653123493513,
 12.318186976450393,
 41.44912128045828,
 24.03010423725024,
 26.314141328295282,
 17.417989657568974,
 22.231285370391305,
 44.98204311479751,
 18.697893506446526,
 23.63535739287989,
 26.170502463943716,
 19.926841227726285,
 17.155605445536715,
 15.45491653952877,
 36.24606292551069,
 17.783202928780515,
 17.40175145231746,
 16.295492856435462,
 15.617303253286888,
 22.018557981542582,
 29.96678182908631,
 23.27498603725865,
 18.259966772372458,
 17.886486018005982,
 18.41216953637679,
 29.62735358842536,
 13.526779404835906,
 31.5684367904448]

In [16]:
mean_rmse = np.mean(rmse_list)
mean_rmse

22.425098894369345

In [17]:
mape_list

[0.6657549050079825,
 0.9759048122535691,
 0.9304201388429593,
 0.795903367096099,
 0.448409204061321,
 0.5201003501919166,
 0.6865877438696724,
 0.3824911656526939,
 0.45035379105800516,
 0.44565868161059097,
 0.6071743986527516,
 1.00604722308183,
 0.6678998879582502,
 0.4640839225060268,
 1.4324518246964157,
 0.540115178954273,
 0.426508663217475,
 0.5391747521743979,
 0.7420533361416649,
 0.829984115644874,
 0.42442099471912775,
 0.9577128272625676,
 0.8696116903441288,
 0.7325662562076837,
 0.6554824548277536,
 0.44011113933857304,
 0.48960515609030253,
 0.4153557920568538,
 0.7839197191755196,
 1.1014284746553116,
 0.699417147221122,
 0.46521777053854035,
 0.34242513884129744,
 0.4002247791547733]

In [18]:
mean_mape = np.mean(mape_list)
mean_mape

0.6568993177384213

###### 